In [1]:
import pandas as pd
import numpy as np
pd.set_option("chained_assignment", None)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)

def load_priorband_data():
    df = pd.read_parquet("data_for_anton.parquet")
    df=df.reset_index()
    df_collection=[]
    for seed_nr in range(50):
        partial_df=df[["benchmark","prior","algorithm","used_fidelity"]]
        partial_df["value"]=df[f"seed-{seed_nr}"]
        partial_df["seed"]=seed_nr
        df_collection.append(partial_df)
        print(f"Seed {seed_nr+1}/50", end="\r", flush=True)
    return pd.concat(df_collection,ignore_index=True)

df=load_priorband_data()

In [2]:
algo_dict={"rs_hb_pb":["random_search","hyperband","pb_mutation_dynamic_geometric-default-at-target"]}
algo_dict["fig7"]=["random_search_prior","bo","bohb","priorband_bo"]
prior_dict={"good+bad":["at25","bad"]}
benchmarks=["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]
label_dict={'random_search': 'RS', 'hyperband': 'HB', 'pb_mutation_dynamic_geometric-default-at-target': 'PB', 'jahs_cifar10': 'JAHS-C10', 'jahs_colorectal_histology': 'JAHS-CH', 'jahs_fashion_mnist': 'JAHS-FM', 'lcbench-126026': 'LC-126026', 'lcbench-167190': 'LC-167190', 'lcbench-168330': 'LC-168330', 'lcbench-168910': 'LC-168910', 'lcbench-189906': 'LC-189906', 'cifar100_wideresnet_2048': 'PD1-Cifar100', 'imagenet_resnet_512': 'PD1-ImageNet', 'lm1b_transformer_2048': 'PD1-LM1B', 'translatewmt_xformer_64': 'PD1-WMT',"random_search_prior":"RS+Prior","bo":"BO","pibo":"PiBO","bohb":"BOHB","priorband_bo":"PriorBand+BO"}

for priortype in df["prior"].unique():
    prior_dict[priortype]=[priortype]


df_dict={}
for prior_key,prior_value in prior_dict.items():
    df_dict[prior_key]={}
    for b_n,benchmark in enumerate(benchmarks):
        print(f"Prior {prior_key}, Benchmark {b_n+1}/{len(benchmarks)}      ", end="\r", flush=True)
        df_dict[prior_key][benchmark]={}
        for algo_key,algo_value in algo_dict.items():
            df_dict[prior_key][benchmark][algo_key]=df.loc[(df["prior"].isin(prior_value))&(df["algorithm"].isin(algo_value))&(df["benchmark"]==benchmark)].reset_index(drop=True)

In [3]:

def create_autorank_incumbent(priors:list[str],fs:list[float],algorithms:list[str],algo_group:str,benchmarks:list[str],df_dict:pd.DataFrame):
    output={}
    output["dataset"]={}
    output["mean_std"]={}
    for p_n,prior in enumerate(priors):
        output["dataset"][prior]={}
        output["mean_std"][prior]={}
        for f_n,max_f in enumerate(fs):
            output["dataset"][prior][max_f]={}
            labels=[]
            for algo in algorithms:
                labels.append(label_dict[algo])
            output["mean_std"][prior][max_f]=pd.DataFrame(columns=["benchmark"]+labels)
            for benchmark in benchmarks:
                output["dataset"][prior][max_f][benchmark]=pd.DataFrame()
                means_std=[]
                for algorithm in algorithms:
                    df_at_point=df_dict[prior][benchmark][algo_group].loc[df_dict[prior][benchmark][algo_group]["algorithm"]==algorithm]
                    df_at_point=df_at_point.loc[df_at_point["used_fidelity"]<=max_f]
                    column=[]
                    for seed in df_at_point["seed"].unique():
                        column.append(df_at_point.loc[df_at_point["seed"]==seed].iloc[-1]["value"])
                    output["dataset"][prior][max_f][benchmark][algorithm]=column
                    means_std.append(str(np.round(output["dataset"][prior][max_f][benchmark][algorithm].mean(),3))+" ± "+str(np.round(output["dataset"][prior][max_f][benchmark][algorithm].std(),3)))
                output["mean_std"][prior][max_f].loc[len(output["mean_std"][prior][max_f])]=[label_dict[benchmark]]+means_std
                print(f"Prior {prior} ({p_n+1}), Fidelity {f_n+1}/{len(fs)}          ", end="\r", flush=True)
                # print(output["mean_std"][prior][max_f].to_markdown(index=False),"\n")
                # print(f"  {max_f}x  {prior}-Prior  {benchmark}")
                # plot_stats(autorank(output["dataset"][prior][max_f][benchmark]),allow_insignificant=True)
                # plt.show()
    return (output["dataset"],output["mean_std"])

In [4]:
table10={}
table10["algo_group"]="rs_hb_pb"
table10["algorithms"]=algo_dict[table10["algo_group"]].copy()
table10["priors"]=["at25","bad","good+bad"]
table10["benchmarks"]=benchmarks
table10["max_f"]=[5,12]

print(create_autorank_incumbent(table10["priors"],table10["max_f"],table10["algorithms"],table10["algo_group"],table10["benchmarks"],df_dict)[1])

{'at25': {5:        benchmark              RS              HB             PB
0       JAHS-C10  11.454 ± 1.935  11.129 ± 1.256  9.986 ± 0.369
1        JAHS-CH   6.752 ± 0.943    6.332 ± 1.12  5.974 ± 0.535
2        JAHS-FM   5.497 ± 0.396   5.805 ± 0.614  5.037 ± 0.021
3      LC-126026   0.053 ± 0.011   0.043 ± 0.008  0.042 ± 0.004
4      LC-167190   0.214 ± 0.021   0.185 ± 0.019  0.193 ± 0.023
5      LC-168330   0.444 ± 0.046   0.408 ± 0.034   0.415 ± 0.02
6      LC-168910    0.38 ± 0.093   0.313 ± 0.019  0.311 ± 0.017
7      LC-189906    0.222 ± 0.09   0.151 ± 0.021   0.151 ± 0.01
8   PD1-Cifar100   0.378 ± 0.156   0.357 ± 0.102  0.258 ± 0.002
9   PD1-ImageNet   0.333 ± 0.108   0.301 ± 0.044  0.224 ± 0.001
10      PD1-LM1B   0.681 ± 0.034   0.651 ± 0.013  0.646 ± 0.008
11       PD1-WMT   0.454 ± 0.097   0.402 ± 0.032   0.37 ± 0.008, 12:        benchmark              RS              HB             PB
0       JAHS-C10  10.451 ± 1.055  10.077 ± 0.769  9.457 ± 0.415
1        JAHS-CH   5.9